# Limpieza de datos con PySpark: Data Science Job Posting on Glassdoor

Los [datos](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/ETYTQ0c-i6FLjM8rZ4iT1cgB6ipFAkainM-4V9M8DXsBiA?e=PeMtvh) fueron extraídos (scrapeados) del sitio web de Glassdoor y recoge los salarios de distintos puestos relacionados a Data.

### Resolver los siguientes requerimientos, para cada operación/moficación imprima como van quedadndo los cambios.

1. Cargar los datos y mostrar el esquema o la informacion de las columnas y el tip de dato de cada columna

In [0]:
#vamos a probar a cargarlo tal cual esta a ver que tenemos (este es el primer csv)
df1 = spark.read.format("csv").option("delimiter", ";").option("header", "true").load("dbfs:/FileStore/shared_uploads/carlosalonsomingo@gmail.com/ds_jobs.csv")

df1.display()

In [0]:
#Vamos a probar el segundo(pasado por el canal de big data)
df2 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/carlosalonsomingo@gmail.com/DS_jobs_in_.csv")
df2.display()

In [0]:
#En vista de que son una locura, vamos a itentar que chatgpt lo normalice un poco
df3 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/carlosalonsomingo@gmail.com/ds_jobs_cleaned.csv")
df3.display()

En vista que no tienen ningun sentido, he buscado patrones en el csv con chatgpt para poder darle algo de sentido a la informacion
[conversación](https://chatgpt.com/share/67488911-d060-8001-8884-558e5a94c4a4)


Aun asi , vemos que tampoco tienemuvho sentido, probablemente , es scrap no tenga demasiada logica de gestion y por ello no tenga mucho sentido, pero vmos a seguir probando(no lo usaré)

In [0]:
#vamos a probar el nuevo csv
df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/carlosalonsomingo@gmail.com/ds_jobs_full_cleaned.csv")
df.display()

Vamos a probar con un codigo siguiendo el analizis de patrones

In [0]:
import pandas as pd
from pyspark.sql import SparkSession

def normalize_csv_pandas_spark(input_path):
    # Leer archivo CSV en Pandas
    data = pd.read_csv(input_path, delimiter=';', encoding='latin1')

    # Limpieza básica de la columna 'Job Description'
    data['Job Description'] = (
        data['Job Description']
        .astype(str)
        .str.replace('', '-', regex=False)  # Reemplazar caracteres especiales
        .str.replace('\n', ' ', regex=False)  # Eliminar saltos de línea
        .str.replace(' +', ' ', regex=True)  # Normalizar espacios múltiples
    )
    
    # Extraer "Duties" de las descripciones
    data['Duties'] = data['Job Description'].str.extract(r'(Duties & Responsibilities:.*?Minimum Qualifications:)')[0]
    data['Duties'] = data['Duties'].str.replace(r'Minimum Qualifications:', '', regex=False).str.strip()

    # Eliminar secciones irrelevantes de la descripción
    data['Job Description'] = data['Job Description'].str.replace(r'Duties & Responsibilities:.*?Minimum Qualifications:', '', regex=True)

    # Limpieza básica de 'Company Name'
    # Separar nombre de la empresa del rating si están juntos (e.g., "CompanyName\n4.5")
    data[['Company Name', 'Rating']] = data['Company Name'].str.extract(r'^(.*?)(?:\n([\d\.]+))?$')
    data['Rating'] = data['Rating'].astype(float)  # Convertir Rating a número

    # Limpieza básica de 'Salary Estimate'
    data['Salary Estimate'] = (
        data['Salary Estimate']
        .str.replace(r'\(Glassdoor est.\)', '', regex=True)
        .str.replace(r'\(Employer est.\)', '', regex=True)
        .str.replace(r'\$', '', regex=False)
        .str.replace(r'K', '000', regex=False)
        .str.strip()
    )
    
    # Limpieza de 'Headquarters'
    data['Headquarters'] = (
        data['Headquarters']
        .astype(str)
        .str.replace('\n', ' ', regex=False)  # Eliminar saltos de línea
        .str.replace(' +', ' ', regex=True)  # Normalizar espacios múltiples
        .str.strip()  # Eliminar espacios iniciales y finales
    )

    # Limpieza de 'Type of ownership'
    data['Type of ownership'] = (
        data['Type of ownership']
        .astype(str)
        .str.replace('\n', ' ', regex=False)  # Eliminar saltos de línea
        .str.strip()  # Eliminar espacios iniciales y finales
    )

    # Limpieza de 'Competitors'
    data['Competitors'] = (
        data['Competitors']
        .astype(str)
        .str.replace('-1', 'Unknown', regex=False)  # Reemplazar valores -1 por "Unknown"
        .str.replace('\n', ', ', regex=False)  # Reemplazar saltos de línea por comas
        .str.strip()  # Eliminar espacios iniciales y finales
    )

    # Filtrar columnas útiles
    columns_to_keep = [
        'Job Title', 'Company Name', 'Rating', 'Salary Estimate', 
        'Location', 'Industry', 'Sector', 'Revenue', 'Size', 
        'Founded', 'Job Description', 'Duties', 'Headquarters',
        'Type of ownership', 'Competitors'
    ]
    data_cleaned = data[columns_to_keep]

    # Crear una sesión de Spark
    spark = SparkSession.builder.appName("Pandas to Spark").getOrCreate()

    # Convertir DataFrame de Pandas a PySpark
    spark_df = spark.createDataFrame(data_cleaned)

    return spark_df

# Uso de la función
# Si estás en Databricks, copia el archivo al sistema local
local_path = "/tmp/ds_jobs.csv"
dbutils.fs.cp("dbfs:/FileStore/shared_uploads/carlosalonsomingo@gmail.com/ds_jobs.csv", f"file:{local_path}")

# Procesar el archivo con Pandas y convertirlo a PySpark
df = normalize_csv_pandas_spark(local_path)

# Mostrar los resultados
df.printSchema()  # Mostrar esquema del DataFrame
df.display()  # Mostrar tabla en Databricks
print(df.count())  # Mostrar el número total de filas


Parece que este ultimo ya tiene mas sentido, usareos este

2. Eliminar duplicados

In [0]:
df = df.distinct()

df.display() #Observamos que de 678 filas pasamos a 661, por lo que hemos eliminado 18 filas duplicadas
print(df.count())

3. Decidir que hacer con los datos faltantes 

4. Decidir que hacer con los valores nulos

In [0]:
#Esta celda tiene en cuenta el apartado 3 y el 4
#primero vemos que columnas contienen nulos y cuantos
from pyspark.sql import functions as F
df_nulls = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns])
df_nulls.display()
#observamos que hay 1 nulo en salary y 659 nulos en dutyes lo que nos dice que no va a aportar mucho esa columna(solo tiene un regitro con info, asique la quitamos)
df = df.drop(F.col("Duties"))
# de momento como todo son strings type, reempalazaremos por no info

df = df.fillna("No info")
df.display()

5. ¿Cuántos registros tiene el csv?

In [0]:
print(f"El csv tiene {df.count()} registros")

6. Mostrar los valores únicos de `Job title` 

In [0]:
df_unique_title = df.select("Job Title").distinct()
df_unique_title.display()

7. Remover la letra `K` de la columna `Salary Estimate` y multiplicar por 1000.

In [0]:
# Podriamos hacerlo multiplicando, pero de momento creo que nos conviene mantener las columnas como string, por lo menos hasta que tengamos que operar con ellas, asi hacemos mas sencillo este paso
df = df.withColumn("Salary Estimate",
                    F.regexp_replace(F.col("Salary Estimate"), "K", "000")
                    )
df.display()

8. Mostrar los valores únicos del campo `Salary Estimate`

In [0]:
df_unique_salary = df.select("Salary Estimate").distinct()
df_unique_salary.display()

9. Eliminar `(Glassdoor est.)` y `(Employer est.)` del campo `Salary Estimate`

In [0]:
df = (df.withColumn("Salary Estimate", F.regexp_replace(F.col("Salary Estimate"), r"\(Glassdoor est.\)", "")) #Para lo de la r"\" lo he buscado ya que no lo ravia, indica que estamos trabajando con texto raw(sin procesar), muy util para las regex que tienen caracteres con significado "
      .withColumn("Salary Estimate", F.regexp_replace(F.col("Salary Estimate"), "\(Employer est.\)", "")))
df.display()

10. Mostrar de mayor a menor los valores del campo `Salary Estimate`

In [0]:
#primero eliminamos el resto de cosas que no son valores

df = df.withColumn("Salary Estimate", F.regexp_replace(F.col("Salary Estimate"), r"\$", ""))
df = df.withColumn("Salary Estimate", F.regexp_replace(F.col("Salary Estimate"), r"K", "000"))
#Ahora separamos para obtener minimo y maximo. Ya que es un rango tomaremos la media para ordenarlos
df_avg_salary_ordered = df.withColumn("avg_salary", (F.split(F.col("Salary Estimate"), "-")[0] + F.split(F.col("Salary Estimate"), "-")[1]) / 2).orderBy(F.col("avg_salary").desc())
df_avg_salary_ordered.display()

11. De la columna `Job Description` quitar los saltos de linea `\n` del texto

In [0]:
df = df.withColumn("Job Description", F.regexp_replace(F.col("Job Description"), r"\n", " "))#Asi seria la consulta, solo que no he metido en el df la descricion por que me daba muchos problemas()
df.display()

12. De la columna `Rating` muestre los valores unicos.

In [0]:
unique_ratings = df.select("Rating").distinct().orderBy("Rating")
unique_ratings.display()

13. Del campo `Rating` reemplazar los `-1.0` por `0.0`.

In [0]:
df = df.withColumn("Rating", F.when(col("Rating") == -1.0, 0.0).otherwise(col("Rating")))
df.display()

14. Mostrar los valores unicos y ordenar los valores del campo `Company Name`.

In [0]:
unique_company_names = df.select("Company Name").distinct().orderBy("Company Name")
unique_company_names.display()


15. Quitar todos los caracteres innecesarios que encuentres en el campo `Company Name`. Por ejemplo los saltos de linea `\n`

In [0]:
df = df.withColumn("Company Name", regexp_replace(col("Company Name"), r"[\n\r]", ""))  # Eliminar saltos de línea
df = df.withColumn("Company Name", regexp_replace(col("Company Name"), r" +", " "))  # Eliminar espacios múltiples
df.display()

16. En el campo `Location` convertir esa columna en dos: `City` y `State`. Las ciudades que tengas en `Location` asignar a la columna `City`. Lo mismo para `State`. Luego elimine la columna `Location`.

In [0]:
df = (df
      .withColumn("City", F.split(F.col("Location"), ", ")[0])
      .withColumn("State", F.split(F.col("Location"), ", ")[1])
      .drop(F.col("Location"))
)
df.display()

17. Repetir la misma lógica de la pregunta 16 pero para el campo `Headquarters`. En Headquarters dejar solo la ciudad, mientras que para el estado añadirla a una columna nueva ` Headquarter State`.

In [0]:
df = (df
      .withColumn("Headquarter State", F.split(F.col("Headquarters"), ", ")[1])
      .withColumn("Headquarters", F.split(F.col("Headquarters"), ", ")[0])
      
)
df.display()

18. Muestre los valores únicos del campo `Headquarter State` 

In [0]:
unique_states = df.select("Headquarter State").distinct()
unique_states.display()

19. Mostrar valores unicos del campo `Size`.

In [0]:
unique_sizes = df.select("Size").distinct().orderBy("Size")
unique_sizes.display()

20. Quitar 'employee' de los registros del campo `Size`. Elimine tambien otros caracteres basura.

In [0]:
from pyspark.sql.functions import regexp_replace, col

# Limpiar el campo 'Size'
df = df.withColumn(
    "Size",
    regexp_replace(col("Size"), r"employees", "")  # Eliminar 'employees'
)
df = df.withColumn(
    "Size",
    regexp_replace(col("Size"), r"[^0-9\-to ]", "")  # Eliminar caracteres basura (excepto números, guiones y 'to')
)
df = df.withColumn(
    "Size",
    regexp_replace(col("Size"), r" +", " ")  # Normalizar espacios múltiples
)

# Mostrar los resultados
df.display()

21. Reemplazar la palabra 'to' por '-' en todos los registros del campo `Size`. Reemplazar tambien '-1' por 'Unknown'. 

In [0]:
df = df.withColumn("Size", regexp_replace(col("Size"), r"\bto\b", "-"))

df = df.withColumn("Size", regexp_replace(col("Size"), r"-1", "Unknown"))
df.display()

22. Mostrar el tipo de dato del campo `Type of ownership` y sus registros unicos.

In [0]:
print("Tipo de dato de 'Type of ownership':", df.schema["Type of ownership"].dataType)
unique_ownerships = df.select("Type of ownership").distinct().orderBy("Type of ownership")
unique_ownerships.display()

23. Cambiar '-1' por 'Unknown' en todos los registros del campo `Type of ownership`.

In [0]:
# Reemplazar '-1' por 'Unknown' en el campo 'Type of ownership'
df = df.withColumn("Type of ownership", F.regexp_replace(F.col("Type of ownership"), r"-1", "Unknown"))

# Mostrar los resultados únicos para validar el cambio
df.select("Type of ownership").distinct().orderBy("Type of ownership").show(truncate=False)

24. Cambiar:  
-  `Company - Public` por `Public Company`  
-  `Company - Private` por `Private Company`  
-  `Private Practice / Firm` por `Private Company`  
-  `Subsidiary or Business Segment` por `Business`  
-  `College / University` por `Education`  
En todos los registros del campo `Type of ownership`.

In [0]:
from pyspark.sql.functions import when, col

# Reemplazar valores en el campo 'Type of ownership'
df = df.withColumn(
    "Type of ownership",
    when(col("Type of ownership") == "Company - Public", "Public Company")
    .when(col("Type of ownership") == "Company - Private", "Private Company")
    .when(col("Type of ownership") == "Private Practice / Firm", "Private Company")
    .when(col("Type of ownership") == "Subsidiary or Business Segment", "Business")
    .when(col("Type of ownership") == "College / University", "Education")
    .otherwise(col("Type of ownership"))  # Mantener el resto sin cambios
)

# Mostrar los resultados únicos para validar los cambios
df.select("Type of ownership").distinct().orderBy("Type of ownership").show(truncate=False)


25. Mostrar el tipo de dato y los valores unicos del campo `Industry`.

In [0]:
# Mostrar el tipo de dato de la columna 'Industry'
print("Tipo de dato de 'Industry':", df.schema["Industry"].dataType)

# Obtener y mostrar los valores únicos de 'Industry'
unique_industries = df.select("Industry").distinct().orderBy("Industry")
unique_industries.show(truncate=False)


26. En el mismo campo de `Industry` reemplazar '-1' por 'Not Available' y '&' por 'and'.  Vuelva a imprimir los valores unicos en orden alfabético.

In [0]:
from pyspark.sql.functions import regexp_replace, col

# Reemplazar '-1' por 'Not Available' y '&' por 'and' en el campo 'Industry'
df = df.withColumn("Industry", regexp_replace(col("Industry"), r"-1", "Not Available"))
df = df.withColumn("Industry", regexp_replace(col("Industry"), r"&", "and"))

# Obtener y mostrar los valores únicos ordenados alfabéticamente
unique_industries = df.select("Industry").distinct().orderBy("Industry")
unique_industries.show(truncate=False)


27. Para el campo `Sector`, muestre el tipo de dato y los valores únicos.

In [0]:
# Mostrar el tipo de dato de la columna 'Sector'
print("Tipo de dato de 'Sector':", df.schema["Sector"].dataType)

# Obtener y mostrar los valores únicos de 'Sector' en orden alfabético
unique_sectors = df.select("Sector").distinct().orderBy("Sector")
unique_sectors.show(truncate=False)


28. Aplica la misma lógica de la pregunta 26 pero sobre el campo `Sector`.

In [0]:
from pyspark.sql.functions import regexp_replace, col

# Reemplazar '-1' por 'Not Available' en el campo 'Sector'
df = df.withColumn("Sector", regexp_replace(col("Sector"), r"-1", "Not Available"))

# Obtener y mostrar los valores únicos de 'Sector' ordenados alfabéticamente
unique_sectors = df.select("Sector").distinct().orderBy("Sector")
unique_sectors.show(truncate=False)


29. Para el campo `Revenue`, muestre el tipo de dato y los valores únicos en orden ascedente.

In [0]:
from pyspark.sql.functions import regexp_replace, col

# Reemplazar '-1' por 'Not Available' en el campo 'Sector'
df = df.withColumn("Sector", regexp_replace(col("Sector"), r"-1", "Not Available"))

# Obtener y mostrar los valores únicos de 'Sector' ordenados alfabéticamente
unique_sectors = df.select("Sector").distinct().orderBy("Sector")
unique_sectors.show(truncate=False)


30. En el campo `Revenue`, cambiar:  
-  `-1` por `N/A`  
-  `Unknown / Non-Applicable` por `N/A`  
-  `Less than $1 million (USD)` por `Less than 1`
-  Quitar `$` y `(USD)`

In [0]:
from pyspark.sql.functions import regexp_replace, col

# Reemplazar '-1' por 'N/A'
df = df.withColumn("Revenue", regexp_replace(col("Revenue"), r"-1", "N/A"))

# Reemplazar 'Unknown / Non-Applicable' por 'N/A'
df = df.withColumn("Revenue", regexp_replace(col("Revenue"), r"Unknown / Non-Applicable", "N/A"))

# Reemplazar 'Less than $1 million (USD)' por 'Less than 1'
df = df.withColumn("Revenue", regexp_replace(col("Revenue"), r"Less than \$1 million \(USD\)", "Less than 1"))

# Quitar '$' y '(USD)'
df = df.withColumn("Revenue", regexp_replace(col("Revenue"), r"\$", ""))
df = df.withColumn("Revenue", regexp_replace(col("Revenue"), r"\(USD\)", ""))

# Mostrar los valores únicos para validar los cambios
unique_revenues = df.select("Revenue").distinct().orderBy("Revenue")
unique_revenues.show(truncate=False)


31. Borrar el campo `Competitors`.

In [0]:
from pyspark.sql.functions import regexp_replace, col

# Modificar los valores del campo 'Revenue'
df = df.withColumn("Revenue", regexp_replace(col("Revenue"), r"-1", "N/A"))  # Reemplazar '-1' por 'N/A'
df = df.withColumn("Revenue", regexp_replace(col("Revenue"), r"Unknown / Non-Applicable", "N/A"))  # Reemplazar 'Unknown / Non-Applicable' por 'N/A'
df = df.withColumn("Revenue", regexp_replace(col("Revenue"), r"Less than \$1 million \(USD\)", "Less than 1"))  # Reemplazar 'Less than $1 million (USD)' por 'Less than 1'
df = df.withColumn("Revenue", regexp_replace(col("Revenue"), r"\$", ""))  # Eliminar '$'
df = df.withColumn("Revenue", regexp_replace(col("Revenue"), r"\(USD\)", ""))  # Eliminar '(USD)'

# Eliminar el campo 'Competitors'
df = df.drop("Competitors")

# Mostrar los resultados únicos del campo 'Revenue' para validar los cambios
df.select("Revenue").distinct().orderBy("Revenue").show(truncate=False)


32. Crear tres columnas: `min_salary` (salario mínimo), `max_salary` (salario maximo) y `avg_salary` (salario promedio) a partir de los datos del campo `Salary Estimate`.

In [0]:
from pyspark.sql.functions import split, col, expr

# Dividir el campo 'Salary Estimate' en min_salary y max_salary
df = df.withColumn("min_salary", split(col("Salary Estimate"), "-")[0].cast("int"))
df = df.withColumn("max_salary", split(col("Salary Estimate"), "-")[1].cast("int"))

# Calcular el promedio del salario
df = df.withColumn("avg_salary", ((col("min_salary") + col("max_salary")) / 2).cast("int"))

# Mostrar las nuevas columnas
df.select("Salary Estimate", "min_salary", "max_salary", "avg_salary").show(truncate=False)


33. Mostrar los valores unicos del campo `Founded` y el tipo de dato.

In [0]:
# Mostrar el tipo de dato de la columna 'Founded'
print("Tipo de dato de 'Founded':", df.schema["Founded"].dataType)

# Obtener y mostrar los valores únicos de 'Founded' ordenados
unique_founded = df.select("Founded").distinct().orderBy("Founded")
unique_founded.show(truncate=False)


34. Reemplazar '-1' por '2024' en todos los registros del campo `Founded`.

In [0]:
from pyspark.sql.functions import regexp_replace, col

# Reemplazar '-1' por '2024' en el campo 'Founded'
df = df.withColumn("Founded", regexp_replace(col("Founded"), r"-1", "2024"))

# Mostrar los resultados únicos para validar el cambio
df.select("Founded").distinct().orderBy("Founded").show(truncate=False)


35. Crear una nueva columna o campo que se llame `company_age` con los datos que se deducen del campo `Founded`.

In [0]:
from pyspark.sql.functions import col, lit, expr

# Calcular la edad de la compañía y crear la columna 'company_age'
df = df.withColumn("company_age", (2024 - col("Founded").cast("int")))

# Manejar casos donde 'Founded' sea '2024' o no esté definido correctamente
df = df.withColumn("company_age", expr("CASE WHEN Founded = '2024' THEN 0 ELSE company_age END"))

# Mostrar los resultados
df.select("Founded", "company_age").show(truncate=False)


36. Crear una columna o campo que se llame: `Job Type` y en cada registro debe ir Senior, Junior o NA según los datos del campo `Job Title`.  
- Cambiar 'sr' o 'senior' o 'lead' o 'principal' por `Senior` en el campo `Job Type`. No olvidar las mayúsculas.
- Cambiar 'jr' o 'jr.' o cualquier otra variante por `Junior`.  
- En cualquier otro caso distinto a los anteriores añadir NA.

In [0]:
from pyspark.sql.functions import when, col, lower

# Crear la columna 'Job Type' basada en las palabras clave del campo 'Job Title'
df = df.withColumn(
    "Job Type",
    when(lower(col("Job Title")).rlike(r"\b(sr|senior|lead|principal)\b"), "Senior")
    .when(lower(col("Job Title")).rlike(r"\b(jr|jr\.)\b"), "Junior")
    .otherwise("NA")
)

# Mostrar los resultados
df.select("Job Title", "Job Type").show(truncate=False)


37. Muestra los registros únicos del campo `Job Type`. 

In [0]:
# Obtener y mostrar los valores únicos del campo 'Job Type'
df.select("Job Type").distinct().orderBy("Job Type").show(truncate=False)


38. Partiendo del campo `Job Description` se extraer todas o las principales skills solicitadas por las empresas, por ejemplo: Python, Spark , Big Data. Cada Skill debe ir en una nueva columna de tipo Binaria ( 0 , 1) o Booleana (True,  False) de modo que cada skill va ser una nueva columna y si esa skill es solicitada por la empresa colocar 1 sino colocar 0. Por ejemplo:  

In [0]:
from pyspark.sql.functions import col, when, lower #No se.... ya estoy desesperado y me falta el cerebro.... llevo 10 horas con este examen....

# Lista de habilidades a extraer
skills = ["Python", "Spark", "Big Data", "SQL", "Machine Learning", "AWS", "TensorFlow", "PyTorch", "R", "Hadoop"]

# Crear una columna para cada habilidad como binaria (0 o 1)
for skill in skills:
    df = df.withColumn(
        skill,
        when(lower(col("Job Description")).contains(skill.lower()), 1).otherwise(0)
    )

# Mostrar las columnas principales junto con las skills
columns_to_display = ["Job Title", "Salary Estimate", "Job Description", "Company Name", "Size", "Founded", "Type of ownership", "Industry", "Sector", "company_age"] + skills
df.select(*columns_to_display).show(truncate=False)
df.display()

Por ejemplo:  
| Job Title         | Salary Estimate | Job Description                                 | Rating | Company Name       | Size       | Founded | Type of ownership         | Industry                       | Sector                         | Same State      | company_age | Python | Excel |
|--------------------|-----------------|-------------------------------------------------|--------|--------------------|------------|---------|---------------------------|--------------------------------|--------------------------------|----------------|-------------|--------|-------|
| Sr Data Scientist | 137000-171000   | Description The Senior Data Scientist is resp... | 3.1    | Healthfirst        | 1001-5000  | 1993    | Nonprofit Organization    | Insurance Carriers            | Insurance Carriers            | Same State      | 31          | 0      | 0     |
| Data Scientist    | 137000-171000   | Secure our Nation, Ignite your Future Join th... | 4.2    | ManTech            | 5001-10000 | 1968    | Public Company            | Research and Development      | Research and Development      | Same State      | 56          | 0      | 0     |
| Data Scientist    | 137000-171000   | Overview Analysis Group is one of the larges... | 3.8    | Analysis Group      | 1001-5000  | 1981    | Private Company           | Consulting                    | Consulting                    | Same State      | 43          | 1      | 1     |
| Data Scientist    | 137000-171000   | JOB DESCRIPTION: Do you have a passion for Da... | 3.5    | INFICON            | 501-1000   | 2000    | Public Company            | Electrical and Electronic Manufacturing | Electrical and Electronic Manufacturing | Different State | 24          | 1      | 1     |


39. Exportar dataset final a csv

40. Extraer todos los insights posibles que sean de valor o utilidad. Cree nuevas columnas, agrupar,  filtrar hacer varios plots que muestren dichos insights que sean de utilidad para una empresa o para un usuario. Elabore conclusiones con los insights encontrados. 

# 1.- análisis salarial


Objetivo: Identificar tendencias salariales, comparar salarios entre diferentes tipos de trabajo, industrias, y tamaños de empresas.

In [0]:
display(df)

# 2.-  Análisis de Habilidades

Objetivo: Identificar las habilidades más demandadas y su relación con el salario.

In [0]:
display(df)

# 3.-  Análisis de Empresa y Tipo de Propiedad

Objetivo: Ver cómo el tipo de propiedad y antigüedad de la empresa afecta el salario y las oportunidades de empleo.

In [0]:
display(df)

# 4.- Distribucion de las ofertas

Objetivo: ver la distribucion de las ofertas segun el tipo de empleo , puesto, experiencia, industria, tamaño...

In [0]:
display(df)

# 5.- Extra: valoracion de las empresas segun diferentes parámetros

Objetivo: asociar la valoracion a diferentes cualidades

In [0]:
display(df)

# Nota importante

No he descrito las conclusiones, pero estaria encantado de contartelas cuando quieras Juanjo, pero no me da ya la cabeza, no puedo mas, he intentado hacer todo el examen por mi mismo, son las 19 de la tarde y ya no me da la cabeza para más, el examen quizas ha sido un poco desmedido, o igual es culpa mia por intentar hacerlo sin apenas ayuda, pero queria medir mis conocimientos de spark sin ayuda de chat-gpt, espero que lo entiendas, pero no puedo mas.
La mayoria de cosas en las que me he ayudado de chatgpt estan anotadas y puedes comprobar en los videos que he hecho muy buen uso de el, no me ha resuelto nada directamente, y te puedo asegurar que en casa igual, me sabe mal no haberle podido dedicar tanto esfuerzo a esta ultima parte ya que era la mas divertida, pero he agotado mi energia en las demás.